EFA for Shape Description
====

In [ ]:
"""
Read in + binarise MNIST
"""

import numpy as np
from sklearn import datasets
import matplotlib.pyplot as plt

n = 5000

data = datasets.fetch_openml("Fashion-MNIST", as_frame=False)

digits = data["data"][:n]
labels = data["target"][:n]

label_meanings = {
    "0": "T-shirt/top",
    "1": "Trouser",
    "2": "Pullover",
    "3": "Dress",
    "4": "Coat",
    "5": "Sandal",
    "6": "Shirt",
    "7": "Sneaker",
    "8": "Bag",
    "9": "Ankle boot",
}

# Pad all the digits slightly
digits = digits.reshape(-1, 28, 28)
digits = 255 * (digits > 32).astype(np.uint8)
digits = np.pad(digits, ((0, 0), (2, 2), (2, 2)), mode="constant", constant_values=0)

assert len(digits) == len(labels)

In [ ]:
"""
Fill holes, otherwise EFA will break (since it prevents us from finding edges)
"""

from tqdm.notebook import tqdm
from scipy.ndimage import binary_fill_holes

from scale_morphology.scales import segmentation

digits = np.array(
    [
        (255 * segmentation.largest_connected_component(binary_fill_holes(d))).astype(
            np.uint8
        )
        for d in tqdm(digits)
    ]
)

In [ ]:
fig, axes = plt.subplots(5, 5)
for axis, digit, label in zip(axes.flat, digits, labels, strict=False):
    im = axis.imshow(digit)
    axis.set_axis_off()
    axis.set_title(label)
fig.tight_layout()

In [ ]:
from skimage import measure
from scale_morphology.scales import efa

fig, axes = plt.subplots(1, 4, figsize=(8, 3))

n_points = 50
orders = [1, 2, 5, 25]


def _raw_coeffs(binary_img, n_points, order):
    """
    Find the Elliptic Fourier expansion coefficients of an object in the given binary image
    without scaling/rotating
    """
    assert measure.euler_number(binary_img) == 1

    x, y = efa.points_around_edge(binary_img, n_points)

    coeffs = efa.pyefd.elliptic_fourier_descriptors(
        np.array([x, y]).T, order=order, normalize=False
    )
    return coeffs


def plot_efd(coeffs, axis, locus, n_points):
    """
    aaaa
    """
    N = coeffs.shape[0]
    N_half = int(np.ceil(N / 2))
    n_rows = 2

    t = np.linspace(0, 1.0, n_points)
    xt = np.ones((n_points,)) * locus[1]
    yt = np.ones((n_points,)) * locus[0]

    for n in range(coeffs.shape[0]):
        xt += (coeffs[n, 0] * np.cos(2 * (n + 1) * np.pi * t)) + (
            coeffs[n, 1] * np.sin(2 * (n + 1) * np.pi * t)
        )
        yt += (coeffs[n, 2] * np.cos(2 * (n + 1) * np.pi * t)) + (
            coeffs[n, 3] * np.sin(2 * (n + 1) * np.pi * t)
        )

    axis.plot(yt - 0.5, xt, "purple", linewidth=2)


example_img = digits[0]

locus = np.mean(np.where(example_img), axis=1)[::-1]
x, y = efa.points_around_edge(example_img, n_points)

for order, axis in zip(orders, axes):
    im = axis.imshow(example_img, origin="upper", cmap="binary", label="Object")
    axis.plot(y, x, "x", color="lime", markersize=5, label="Points on edge")
    coeffs = _raw_coeffs(example_img, n_points, order)

    plot_efd(coeffs, axis, locus, n_points)

    axis.set_axis_off()
    axis.set_title(f"Order {order}")
fig.tight_layout()

Power Spectrum
----
Sum of squared coefficients - rapidly drops off with order

In [ ]:
"""
Fit to some shapes, get the power spectra, plot them

"""


def power(coeffs):
    return np.sum(coeffs**2, axis=1)


n = 100
order = 20
powers = [power(efa.coefficients(d, n_points, order)) for d in tqdm(digits[:n])]

median_power = np.median(powers, axis=0)
quartiles = np.quantile(powers, [0.25, 0.75], axis=0)

fig, axis = plt.subplots()
o = np.arange(order)
axis.plot(o, median_power, "k")
axis.fill_between(o, *quartiles, alpha=0.3, color="k")
axis.set_xlabel("Harmonic Order")
axis.set_ylabel("Power /arbitrary units")

Clustering/classification
----

In [ ]:
"""
Find the EFA coefficients, up to a certain order, now rotated to remove the major-axis dependence
"""


order = 50

coeffs = [efa.coefficients(d, n_points, order) for d in tqdm(digits)]

In [ ]:
"""
Perform PCA
"""

from sklearn.decomposition import PCA

flat_coeffs = np.array(coeffs).reshape(-1, order * 4)

pca = PCA(n_components=2)
pca_transformed = pca.fit_transform(flat_coeffs)

In [ ]:
fig, axis = plt.subplots(figsize=(10, 10))
for label in np.unique(labels):
    keep = labels == label
    axis.scatter(*(pca_transformed[keep].T), label=label_meanings[label], alpha=0.5)

axis.legend()
fig.tight_layout()

In [ ]:
"""
Do LDA instead
"""

from sklearn.discriminant_analysis import LinearDiscriminantAnalysis

n_plot_dim = 2
lda = LinearDiscriminantAnalysis(n_components=n_plot_dim)
lda_transformed = lda.fit_transform(flat_coeffs, labels)
score = lda.score(flat_coeffs, labels)
[x for x in dir(lda) if x.endswith("_") and not x.startswith("_")]

In [ ]:
fig, axis = plt.subplots(figsize=(8, 8))
for label in np.unique(labels):
    keep = labels == label
    axis.scatter(*(lda_transformed[keep].T), label=label_meanings[label], alpha=0.5)

axis.legend()
axis.set_title(f"Accuracy: {100*score:.2f}%; {n_plot_dim} components shown")
var1, var2 = lda.explained_variance_ratio_
axis.set_xlabel(f"Explains {100*var1:.2f}% variance")
axis.set_ylabel(f"Explains {100*var2:.2f}% variance")
fig.tight_layout()

As a conjugate space
----
### Area

In [ ]:
"""
We can calculate the area directly from the (unscaled) coefficients
"""


def area(coeffs: np.ndarray):
    a, b, c, d = coeffs.T
    ns = np.arange(len(a)) + 1
    return -np.pi * np.sum(ns * (a * d - b * c))


areas = [np.sum(d) / 255 for d in digits]
approx_areas = [area(c) for c in tqdm(coeffs)]

fig, axis = plt.subplots()
axis.plot(areas, approx_areas, "k.", markersize=0.5)
axis.set_xlabel("True area")
axis.set_ylabel("Area from EFA coeffs")
axis.set_title(f"r2 = {np.corrcoef(approx_areas, areas)[0, 1]:.6f}")

### Alignment with first ellipse

In [ ]:
"""
We can align two images using the (un-rotated) coefficients
"""

from scipy.ndimage import rotate

angle = 45

img = digits[0]
rotated_img = rotate(img, angle, order=0, reshape=False)

# Set edge pixels to 0
rotated_img[0] = 0
rotated_img[-1] = 0
rotated_img[:, 0] = 0
rotated_img[:, -1] = 0

fig, axes = plt.subplots(1, 2, sharex=True, sharey=True)
for a, i in zip(axes, [img, rotated_img]):
    a.imshow(i, cmap="binary")
    a.set_axis_off()

fig.suptitle("Similar images, but rotated")
fig.tight_layout(rect=[0, 0.3, 1, 0.95])

In [ ]:
fig, axes = plt.subplots(1, 2)
n_points = 50


def plot_axis(axis, coeffs):
    assert coeffs.shape == (1, 4)

    ((a, b, c, d),) = coeffs

    theta = 0.5 * np.arctan2(2 * (a * b + c * d), (a**2 - b**2 + c**2 - d**2))

    A = np.sqrt(a**2 + b**2)
    C = np.sqrt(c**2 + d**2)
    if A < C:
        length = C
    else:
        length = A
    theta += np.pi / 2

    dx, dy = length * np.cos(theta), length * np.sin(theta)
    axis.plot(
        [locus[0] - dx, locus[0] + dx],
        [locus[1] - dy, locus[1] + dy],
    )
    return theta


thetas = []
for axis, i in zip(axes, (img, rotated_img)):
    axis.imshow(i, origin="upper", cmap="binary")

    locus = np.mean(np.where(i), axis=1)[::-1]
    axis.scatter(*locus)
    x, y = efa.points_around_edge(i, n_points)

    axis.plot(y, x, "x", color="lime", markersize=5)
    coeffs = _raw_coeffs(i, n_points, 1)

    plot_efd(coeffs, axis, locus, n_points)

    theta = plot_axis(axis, coeffs)
    thetas.append(theta)

    axis.set_title(f"Angle {180 - 180 * theta/np.pi:.1f}" r"$\degree$")
    axis.set_axis_off()

In [ ]:
re_rotated = rotate(
    rotated_img, (180 * (thetas[1] - thetas[0]) / np.pi), reshape=False, order=0
)

fig, axes = plt.subplots(1, 3)
for axis, i, label in zip(
    axes, [rotated_img, img, re_rotated], ["Rotated", "Original", "Corrected"]
):
    axis.imshow(i, cmap="binary")
    axis.set_axis_off()
    axis.set_title(label)
fig.tight_layout()

### Alignment with SVD
We can formulate the pointwise distance, and write this in terms of the EFA coefficients to get a term for distance in terms of our coeffs.
Then, we can minimise this distance by applying a rotation matrix to one set of coeffs - the optimal rotation matrix will align our curves.

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(10, 5), sharex=True, sharey=True)

img = digits[15]
rotated_img = rotate(img, angle, order=0, reshape=False)

# Set edge pixels to 0
rotated_img[0] = 0
rotated_img[-1] = 0
rotated_img[:, 0] = 0
rotated_img[:, -1] = 0

tmp = []
loci = []
for axis, i in zip(axes[0], [img, rotated_img]):
    axis.imshow(i, cmap="binary")

    locus = np.mean(np.where(i), axis=1)[::-1]
    x, y = efa.points_around_edge(i, n_points)

    axis.plot(y, x, "x", color="lime", markersize=5)
    coeffs = _raw_coeffs(i, n_points, 25)

    plot_efd(coeffs, axis, locus, n_points)
    axis.set_axis_off()

    tmp.append(coeffs)
    loci.append(locus)

original_coeffs, rotated_coeffs = tmp

for axis, c, locus in zip(axes[1], tmp, loci):
    plot_efd(c, axis, locus, n_points)
    axis.set_axis_off()

In [ ]:
"""
We can rotate the coefficients of one onto the other by taking the sum of the coefficient matrices and performing SVD
"""

orig_matrices = np.transpose(original_coeffs.reshape(-1, 2, 2), axes=(0, 2, 1))
rotated_matrices = rotated_coeffs.reshape(-1, 2, 2)

S = np.einsum("ijk,ilk->ijl", orig_matrices, rotated_matrices).sum(axis=(0))

In [ ]:
from scipy.linalg import svd

U, _, Vt = svd(S)
rot_matrix = np.transpose(Vt) @ np.transpose(U)

In [ ]:
# Apply this rotation matrix to the rotated matrices
new_coeffs = np.einsum("ij,kjl->kil", rot_matrix, rotated_matrices).reshape(-1, 4)

In [ ]:
original_coeffs, rotated_coeffs = tmp

fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True)

for axis, c, title in zip(
    axes,
    (original_coeffs, rotated_coeffs, new_coeffs),
    ("Original", "Rotated", "Corrected with EFA rotation"),
):
    plot_efd(c, axis, (0, 0), n_points)
    axis.set_axis_off()
    axis.set_aspect("equal")
    axis.set_ylim(15, -15)
    axis.set_title(title)

fig.tight_layout()

### Rotation with weighting

In [ ]:
p = power(original_coeffs)
new_s = np.einsum("i,ijk,ilk->ijl", p, orig_matrices, rotated_matrices).sum(axis=0)

U, _, Vt = svd(S)
rot_matrix = (U @ Vt).T
new_coeffs = np.einsum("ij,kjl->kil", rot_matrix, rotated_matrices).reshape(-1, 4)

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(9, 3), sharex=True)

for axis, c, title in zip(
    axes,
    (original_coeffs, rotated_coeffs, new_coeffs),
    ("Original", "Rotated", "Corrected with EFA rotation\n(power scaled)"),
):
    plot_efd(c, axis, (0, 0), n_points)
    axis.set_axis_off()
    axis.set_aspect("equal")
    axis.set_ylim(15, -15)
    axis.set_title(title)

fig.tight_layout()